In [1]:
# WIP Doesn't Work Yet

import requests

url = ""

payload = {"page_size": 100}
headers = {
    "accept": "application/json",
    "Notion-Version": "2022-06-28",
    "content-type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)

{"object":"error","status":401,"code":"unauthorized","message":"API token is invalid."}
